<a href="https://colab.research.google.com/github/mike1336git/colab_notebook/blob/main/with_js/js027_NaClIonMD3D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### simulator( html + css + js ) + control( python )

In [2]:
#@title js027_NaClIonMD3D / def exec_html_js() ... exec me first
#
#  Copyright(C) 2023-2024 Mitsuru Ikeuchi
#  home page: https://mike1336.web.fc2.com/index.html
#  Released under the MIT license ( https://opensource.org/licenses/MIT )
#
#  ver 0.0.0  2023.09.22 created,  last updated on 2024.05.04
#

# def exec_html_js()

import IPython
from IPython.display import display, HTML
from google.colab.output import eval_js

def exec_html_js():
  htm = HTML('''


<!DOCTYPE html>
<html>
<head>
<meta charset="utf-8">
<title>js027_NaClIonMD3D</title>
<script type="text/javascript">

// %%%%%%%%%%%%%%%%%%%%  javaScript  %%%%%%%%%%%%%%%%%%%%

'use strict';

/* --------------------
//
//  js027_NaClIonMD3D
//    Copyright(C) 2017-2023 Mitsuru Ikeuchi
//    Released under the MIT license ( https://opensource.org/licenses/MIT )
//
//    ver 0.0.0  2017.05.14 created, last updated on 2018.11.17
//    ver 0.0.1  2019.01.15 v1, last updated on 2021.05.16
//    ver 0.0.2  2021.10.30 v2, last updated on 2021.10.30
//    ver 0.0.3  2023.03.12 v3, last updated on 2023.08.20
//
// --------------------  molecular dynamics 2D
//
//    method: velocity Verlet Algorithm
//      (1) vi = vi + (Fi/mi)*(0.5dt)
//      (2) ri = ri + vi*dt
//      (3) calculation Fi <- {r1,r2,...,rn} Fi=sum(Fij,j=1 to n),Fij=F(ri-rj)
//      (4) vi = vi + (Fi/mi)*(0.5dt)
//      goto (1)
//
//    force: ion f(r) = fc + fr + fa,  fc:Debye-screened Coulomb force
//      fc = eForceConst*zi*zj*(EXP(-r/6.5e-10)/r)*(1.0/r+1.0/6.5e-10)
//      fr = 6.9742e-11*EXP((a-r)/b)    :repulsive force
//      fa = -6.9742e-21*(c/r^6)        :attractive force
//
//    for fast calculation
//      ignore F(r) r>rCutoff
//      force F(r) <- force table + linear interpolation (see setForceTable() and cutoff(r))
//      register near particles reg[][] (see registerNearParticles()), 'near' means r<rCutoff+20*2000*dt
//        reg[][] use 20 times, assuming particle max speed < 2000m/s
//      force calculation: sum up force(r) (r<rCutoff)
//
// --------------------
*/

const NaClIonMD3D = (function(){ // ====================  NaClIonMD3D Module  ====================

	const g_AMU = 1.66053904e-27;				// (kg) atomic mass unit
	const g_kB = 1.380649e-23;					// (J/K) Boltzmann's constant
	const g_EE = 1.602176634e-19;				// (C) electron charge, energy : 1(eV) = EE(J)
	const g_eForceConst = 1.0/(4.0*Math.PI*8.85418782e-12); // epsilon0=8.85418782e-12
	const g_nMax = 8000;						// array max

	let g_ionKind1 = 3;							// 3:Na+
	let g_ionKind2 = 7;							// 7:Cl-
	let g_nParticles =100;						// number of particles
	let g_sysTime = 0.0;						// (s) system time
	let g_timeStep =  2.0*1.0e-15;				// (s) time step dt
	let g_xMax = 6.0E-9;						// (m) x-Box size
	let g_yMax = 6.0E-9;						// (m) y-Box size
	let g_zMax = 6.0E-9;						// (m) z-Box size
	let g_rCutoff = 1.2e-9;						// (m) force cutoff length, *** rCutoff = 1.0e-9 can not reproduce
	let g_hh = 1.0e-12;							// (m) forceTable r-division
	let g_kineticEnergy = 0.0;					// (J) total kinetic energy
	let g_potentialEnergy = 0.0;				// (J) total potential energy
	let g_sysTemp = 0.0;						// (K) system temperature
	let g_meanTemp = 0.0;						// (K) averaged system temperature

	const g_xx = dim1( g_nMax );				// (m) x-component of i-th particle position
	const g_yy = dim1( g_nMax );				// (m) y-component of i-th particle position
	const g_zz = dim1( g_nMax );				// (m) z-component of i-th particle position
	const g_vx = dim1( g_nMax );				// (m/s) x-component of i-th particle velocity
	const g_vy = dim1( g_nMax );				// (m/s) y-component of i-th particle velocity
	const g_vz = dim1( g_nMax );				// (m/s) z-component of i-th particle velocity
	const g_ffx = dim1( g_nMax );				// (N) x-component of total force applied i-th particle
	const g_ffy = dim1( g_nMax );				// (N) y-component of total force applied i-th particle
	const g_ffz = dim1( g_nMax );				// (N) z-component of total force applied i-th particle
	const g_mas = dim1( g_nMax );				// (kg) mass of i-th particle
	const g_kind = dimInt1( g_nMax );			// kind of i-th particle
	const g_potentialTable = dim3( 18, 18, 1502 );// potential table [V[0], V[hh], V[2hh],..., V[rCutoff] ]
	const g_forceTable = dim3( 18, 18, 1502 );	// force table [F[0], F[hh], F[2hh],..., F[rCutoff] ]
	const g_reg = dimInt2( g_nMax, 670 );		// reg[i][0]:total number of particles near i-th particle
												// reg[i][j]:particle number near i-th particle, (j>0)

	function dim1( n ) {
		return new Float64Array( n );
	}

	function dimInt1( n ) {
		return new Int32Array( n );
	}

	function dimInt2( ni, nj ) {
		let a = [];
		for (let i=0; i<ni; i++) {
			a[i] = new Int32Array( nj );
		}
		return a;
	}

	function dim3( ni, nj, nk ) {
		let a = [];
		for (let i=0; i<ni; i++) {
			a[i] = [];
			for (let j=0; j<nj; j++) {
				a[i][j] = new Float64Array( nk );
			}
		}
		return a;
	}


	// --------------------  SXion potential data  --------------------

	//   0 mass      , 1 charge , 2 a      , 3 b      , 4 c, 5 r-ion  ,  6 color , 7 str
	const g_SXion = [
		[ 10.81*g_AMU,  3.0*g_EE, 0.720e-10, 0.080e-10,  0.0, 0.23e-10, "#ff4060", "B+++"  ], //  0
		[ 16.00*g_AMU, -2.0*g_EE, 1.626e-10, 0.085e-10, 20.0, 1.40e-10, "#4000ff", "O--"   ], //  1
		[ 19.00*g_AMU, -1.0*g_EE, 1.565e-10, 0.085e-10, 20.0, 1.33e-10, "#0040ff", "F-"    ], //  2
		[ 22.99*g_AMU,  1.0*g_EE, 1.260e-10, 0.080e-10, 20.0, 1.02e-10, "#ff0000", "Na+"   ], //  3
		[ 24.31*g_AMU,  2.0*g_EE, 1.161e-10, 0.080e-10, 10.0, 0.72e-10, "#ff6000", "Mg++"  ], //  4
		[ 26.98*g_AMU,  3.0*g_EE, 1.064e-10, 0.080e-10,  2.0, 0.53e-10, "#ff4080", "Al+++" ], //  5
		[ 28.09*g_AMU,  4.0*g_EE, 1.012e-10, 0.080e-10,  0.0, 0.40e-10, "#ff8040", "Si++++"], //  6
		[ 35.45*g_AMU, -1.0*g_EE, 1.950e-10, 0.090e-10, 30.0, 1.81e-10, "#0000ff", "Cl-"   ], //  7
		[ 39.10*g_AMU,  1.0*g_EE, 1.595e-10, 0.080e-10, 15.0, 1.38e-10, "#ff4000", "K+"    ], //  8
		[ 40.08*g_AMU,  2.0*g_EE, 1.414e-10, 0.080e-10, 10.0, 1.00e-10, "#ff8000", "Ca++"  ], //  9
		[ 47.88*g_AMU,  4.0*g_EE, 1.235e-10, 0.080e-10,  0.0, 0.61e-10, "#ff8060", "Ti++++"], // 10
		[ 87.62*g_AMU,  2.0*g_EE, 1.632e-10, 0.080e-10, 15.0, 1.16e-10, "#ffaf00", "Sr++"  ], // 11
		[ 137.3*g_AMU,  2.0*g_EE, 1.820e-10, 0.080e-10, 20.0, 1.36e-10, "#ffbf00", "Ba++"  ], // 12
		[ 4.003*g_AMU,  0.0*g_EE, 1.200e-10, 0.110e-10, 4.76, 1.28e-10, "#00ff80", "He"    ], // 13
		[ 20.18*g_AMU,  0.0*g_EE, 1.415e-10, 0.112e-10,11.03, 1.37e-10, "#00ff40", "Ne"    ], // 14
		[ 39.95*g_AMU,  0.0*g_EE, 1.878e-10, 0.117e-10,38.53, 1.70e-10, "#00ff00", "Ar"    ], // 15
		[ 83.80*g_AMU,  0.0*g_EE, 2.041e-10, 0.130e-10,55.33, 1.83e-10, "#40ff00", "Kr"    ], // 16
		[ 131.3*g_AMU,  0.0*g_EE, 2.258e-10, 0.145e-10,85.55, 1.99e-10, "#80ff00", "Xe"    ] ];//17

	const g_rCollision = [];// (m) V(r) = 0 at r==g_rCollision[kind]
	const g_rBond = [];		// (m) bpnd length == 2.0*g_rBond[kind]
	const g_massOf = [];	// (kg) g_massOf[kind] : mass of kind
	const g_strOf = [];		// g_strOf[kind] : string of kind, such as "Fe"
	const g_colorOf = [];	// g_colorOf[kind] : color of kind

	(function() {
		const n=g_SXion.length;

		for (let i=0; i<n; i++) {
			g_rCollision[i] = g_SXion[i][5]/1.12246;
			g_rBond[i] = g_SXion[i][5];
			g_massOf[i] = g_SXion[i][0];
			g_strOf[i] = g_SXion[i][7];
			g_colorOf[i] = g_SXion[i][6];
		}
	}());

	//                      0     1     2    3     4     5     6     7     8    9
	const g_strOfAtom = [  "B",  "O",  "F", "Na", "Mg", "Al", "Si", "Cl", "K", "Ca",
						"Ti", "Sr", "Ba", "He", "Ne", "Ar", "Kr", "Xe" ];

	function colorStrOf(kind) {
		return "<span style='color:"+g_colorOf[kind]+"'>"+g_strOf[kind]+"</span>";
	}

	//--- set forceTable[kindi][kindj][ir] and potentialTable[kindi][kindj][ir]

	function setForceTable() {
		const hh=g_hh;
		for (let ki=0; ki<18; ki++) {
			for (let kj=0; kj<18; kj++) {
				const a = g_SXion[ki][2]+g_SXion[kj][2];
				const b = g_SXion[ki][3]+g_SXion[kj][3];
				const c = g_SXion[ki][4]*g_SXion[kj][4]*1.0e-60;
				const zi = g_SXion[ki][1];
				const zj = g_SXion[kj][1];
				for (let ir=1; ir<=1501; ir++) {
					const r = ir*hh;
					const fc = g_eForceConst*zi*zj*(Math.exp(-r/6.5e-10)/r)*(1.0/r+1.0/6.5e-10);
					const fr = 6.9742e-11*Math.exp((a-r)/b); //repulsive force;
					const fa = -6.9742e-21*(c/(r*r*r*r*r*r)); //attractive force;
					g_forceTable[ki][kj][ir] = cutoff(r)*(fc + fr + fa);
				}
				g_forceTable[ki][kj][0] = g_forceTable[ki][kj][1] + g_forceTable[ki][kj][2];
				g_potentialTable[ki][kj][1501] = 0.0;
				for (let ir=1500; ir>=0; ir--) {
					g_potentialTable[ki][kj][ir] = g_potentialTable[ki][kj][ir+1]+g_forceTable[ki][kj][ir]*hh;
				}
			}
		}
	}

	function cutoff(r) {
		let ret;
		if (r>0 && r<0.8*g_rCutoff) {
			ret = 1.0;
		} else if (r>=0.8*g_rCutoff && r<g_rCutoff ) {
			ret = 0.5+0.5*Math.cos(Math.PI*(r-0.8*g_rCutoff)/(0.2*g_rCutoff));
		} else {
			ret = 0.0;
		}
		return ret;
	}


	// --------------------  set initial condition  --------------------

	function setInitialCondition( material, boxSizeInNM, contTemp ) {
		const n=3;
		g_sysTime = 0.0;
		setForceTable();
		g_xMax = boxSizeInNM*1.0e-9;
		g_yMax = boxSizeInNM*1.0e-9;
		g_zMax = boxSizeInNM*1.0e-9;
		let lattice;
		if (material==0) { // NaCl
			g_ionKind1 = 3; // Na+
			g_ionKind2 = 7; // Cl-
			lattice = 5.6407e-10;
		} else if (material==1) { // MgO
			g_ionKind1 = 4; // Mg++
			g_ionKind2 = 1; // O--
			lattice = 4.212e-10*0.97; //0.97: correction factor
		} else if (material==2) { // CaO
			g_ionKind1 = 9; // Ca++
			g_ionKind2 = 1; // O--
			lattice = 4.80e-10*0.97; //0.97: correction factor
		} else if (material==3) { // BaO
			g_ionKind1 = 12; // Ba++
			g_ionKind2 = 1;  // O--
			lattice = 5.536e-10;
		} else if (material==4) { // NaF
			g_ionKind1 = 3;  // Na+
			g_ionKind2 = 2;  // F-
			lattice = 4.62e-10;
		} else if (material==5) { // KF
			g_ionKind1 = 8;  // K+
			g_ionKind2 = 2;  // F-
			lattice = 5.34e-10;
		} else if (material==6) { // KCl
			g_ionKind1 = 8;  // K+
			g_ionKind2 = 7;  // Cl-
			lattice = 6.29e-10;
		}
		const s = 0.5*(g_xMax - lattice*n);
		g_nParticles = setNaClTypeBlock(0, g_ionKind1, g_ionKind2, n, n, n, lattice, s, s, s);
		ajustVelocity(contTemp);
	}

	function setNaClTypeBlock2D(ii, knd1, knd2, nx, ny, lattice, xPos, yPos) { // no use
		let ipp = ii;
		const a = lattice/2.0;
		for (let i=0; i<2*nx; i++) {
			for (let j=0; j<2*ny; j++) {
				const x = xPos + a*i;
				const y = yPos + a*j;
				let knd;
				if ((i+j)%2==0) {
					knd = knd1;
				} else {
					knd = knd2;
				}
				setParticle(ipp, knd, x, y,0.5*g_zMax);
				ipp = ipp + 1;
			}
		}
		return ipp;
	}

	function setNaClTypeBlock(ii, knd1, knd2, nx, ny, nz, lattice, xPos, yPos, zPos) {
		let ipp = ii;
		const a = lattice/2.0;
		for (let i=0; i<2*nx; i++) {
			for (let j=0; j<2*ny; j++) {
				for (let k=0; k<2*nz; k++) {
					const x = xPos + a*i;
					const y = yPos + a*j;
					const z = zPos + a*k;
					let knd;
					if ((i+j+k)%2==0) {
						knd = knd1;
					} else {
						knd = knd2;
					}
					setParticle(ipp, knd, x, y, z);
					ipp = ipp + 1;
				}
			}
		}
		return ipp;
	}

	function setParticle(i, knd, x, y, z) {
		g_xx[i] = x;
		g_yy[i] = y;
		g_zz[i] = z;
		g_vx[i] = 200.0*normalRandom3();
		g_vy[i] = 200.0*normalRandom3();
		g_vz[i] = 200.0*normalRandom3();
		g_ffx[i] = 0.0;
		g_ffy[i] = 0.0;
		g_ffz[i] = 0.0;
		g_kind[i] = knd;
		g_mas[i] = g_massOf[knd];
	}

	// normal distributed random number: -3.0 <= normalRandom3() < 3.0
	function normalRandom3() {
		return (Math.random()+Math.random()+Math.random()+Math.random()+Math.random()+Math.random()-3.0);
	}


	// --------------------  time evolution  --------------------

	function timeEvolution( tempMode, contTemp ) {
		const nn=g_nParticles;
		if (tempMode==1) ajustVelocity(contTemp);
		registerNearParticles();
		for (let i=0; i<20; i++) {
			g_sysTime += g_timeStep;
			moveParticles(g_timeStep);
		}
		g_kineticEnergy = 0.0;
		for (let i=0; i<nn; i++) {
			g_kineticEnergy += 0.5*g_mas[i]*(g_vx[i]*g_vx[i]+g_vy[i]*g_vy[i]+g_vz[i]*g_vz[i]);
		}
		g_sysTemp = 2.0*g_kineticEnergy/(3.0*nn*g_kB); //2D: E/N=kT, 3D: E/N=(3/2)kT
		g_meanTemp = 0.95*g_meanTemp + 0.05*g_sysTemp;
	}

	function moveParticles(dt) {
		const nn=g_nParticles;
		for (let i=0; i<nn; i++) {
			const a = 0.5*dt/g_mas[i];
			g_vx[i] += a*g_ffx[i];
			g_vy[i] += a*g_ffy[i];
			g_vz[i] += a*g_ffz[i];
			g_xx[i] += g_vx[i]*dt;
			g_yy[i] += g_vy[i]*dt;
			g_zz[i] += g_vz[i]*dt;
		}
		calcForce();
		for (let i=0; i<nn; i++) {
			const a = 0.5*dt/g_mas[i];
			g_vx[i] += a*g_ffx[i];
			g_vy[i] += a*g_ffy[i];
			g_vz[i] += a*g_ffz[i];
		}
	}

	function calcForce() {
		const nn=g_nParticles, s05 = 0.5*3.418e-10; // Ar sigma=3.418e-10
		g_potentialEnergy = 0.0;
		for (let i=0; i<nn; i++) {
			g_ffx[i]=0;g_ffy[i]=0;g_ffz[i]=0;
		}
		for (let i=0; i<nn-1; i++) {
			for (let k=1; k<g_reg[i][0]; k++) {
				const j = g_reg[i][k];
				const xij=g_xx[i]-g_xx[j], yij=g_yy[i]-g_yy[j], zij=g_zz[i]-g_zz[j];
				const rij = Math.sqrt(xij*xij+yij*yij+zij*zij);
				if (rij<g_rCutoff) {
					const f = force(rij,g_kind[i],g_kind[j]);
					const fxij = f*xij/rij;
					const fyij = f*yij/rij;
					const fzij = f*zij/rij;
					g_ffx[i] += fxij;
					g_ffy[i] += fyij;
					g_ffz[i] += fzij;
					g_ffx[j] -= fxij;
					g_ffy[j] -= fyij;
					g_ffz[j] -= fzij;
				}
			}
		}
		for (let i=0; i<nn; i++) {
			g_ffx[i] += boundaryForce(g_xx[i]+s05)+boundaryForce(g_xx[i]-g_xMax-s05);
			g_ffy[i] += boundaryForce(g_yy[i]+s05)+boundaryForce(g_yy[i]-g_yMax-s05);
			g_ffz[i] += boundaryForce(g_zz[i]+s05)+boundaryForce(g_zz[i]-g_zMax-s05);
		}
	}

	function force(r,kndi,kndj) { // forceTable - linear interporation
		const hh=g_hh;
		const ir = Math.floor(r/hh);
		const a = r - ir*hh;
		g_potentialEnergy += ((hh-a)*g_potentialTable[kndi][kndj][ir] + a*g_potentialTable[kndi][kndj][ir+1])/hh;
		return ((hh-a)*g_forceTable[kndi][kndj][ir] + a*g_forceTable[kndi][kndj][ir+1])/hh;
	}

	function boundaryForce(r) {
		const ri = (3.418e-10/r);
		const r6 = ri*ri*ri*ri*ri*ri;
		g_potentialEnergy += 4.0*0.5*1.711e-21*r6*(r6-1.0); // epsilonOfAr = 1.711e-21
		return (24.0*0.5*1.711e-21*r6*(2.0*r6-1.0)/r);
	}

	//--- registeration

	function registerNearParticles() {
		const nn=g_nParticles;
		const rCut = g_rCutoff+20*2000*g_timeStep;
		const rcut2 = rCut*rCut;
		for (let i=0; i<nn-1; i++) {
			let k = 1;
			for (let j=i+1; j<nn; j++) {
				const r2 = (g_xx[i]-g_xx[j])*(g_xx[i]-g_xx[j])+(g_yy[i]-g_yy[j])*(g_yy[i]-g_yy[j])
						+(g_zz[i]-g_zz[j])*(g_zz[i]-g_zz[j]);
				if (r2<rcut2) {
					g_reg[i][k] = j;
					k = k + 1;
				}
			}
			g_reg[i][0] = k;
		}
	}

	function maxNearParticles() {
		const nn=g_nParticles;
		let mx=0;
		for (let i=0; i<nn-1; i++) {
			if (mx<g_reg[i][0]) mx = g_reg[i][0];
		}
		return (mx-1);
	}


	// --------------------  utility  --------------------

	function systemTemperature() {
		const nn=g_nParticles;
		let ek=0.0;  //kinetic energy (J)
		for (let i=0; i<nn; i++) {
			ek = ek + 0.5*g_mas[i]*(g_vx[i]*g_vx[i]+g_vy[i]*g_vy[i]+g_vz[i]*g_vz[i]);
		}
		return 2.0*ek/(3.0*nn*g_kB); //2D: E/N=kT, 3D: E/N=(3/2)kT
	}

	function ajustVelocity(temp) {
		const nn=g_nParticles;
		const a = Math.sqrt(temp/systemTemperature());
		for (let i=0; i<nn; i++) {
			g_vx[i] = a*g_vx[i];
			g_vy[i] = a*g_vy[i];
			g_vz[i] = a*g_vz[i];
		}
	}


	// --------------------  public  --------------------

	return {
		init:			setInitialCondition,	// setInitialCondition( material, boxSizeInNM, contTemp)
		evolve:			timeEvolution,			// timeEvolution( tempMode, contTemp )

		setTemp:		ajustVelocity,			// ajustVelocity( temp )

		getSysParam:	function() {
							return [ g_ionKind1, g_ionKind2, g_nParticles, g_timeStep, g_xMax, g_yMax, g_zMax ]; },
		getNow:			function() { return [ g_sysTime, systemTemperature(), g_kineticEnergy, g_potentialEnergy ]; },
		getKindStr:		function(kind) { return g_strOfAtom[kind]; },

		getBondLength:	function(i,j) { return ( g_rBond[g_kind[i]] + g_rBond[g_kind[j]] ); },
		getChargeProduct: function(i,j) { return g_SXion[g_kind[i]][1]*g_SXion[g_kind[j]][1]; },
		getNearList:	function(i) { return g_reg[i]; },
		getIonData:		function(i) { return [ g_kind[i], g_rCollision[g_kind[i]], g_rBond[g_kind[i]] ]; },

		getPosition:	function(i) { return [ g_xx[i], g_yy[i], g_zz[i] ]; },
		getVelocity:	function(i) { return [ g_vx[i], g_vy[i], g_vz[i] ]; },
		getForce:		function(i) { return [ g_ffx[i], g_ffy[i], g_ffz[i] ]; },
	};

})(); // ====================  NaClIonMD3D end  ====================


const js027 = (function(){ // ====================  js Module  ====================

	const theModule = NaClIonMD3D;
	const xCanvasSize = 480;	// in pixel
	const yCanvasSize = 480;	// in pixel

	let canvas;
	let ctx;

	let v_theme = 0;
	let v_BoxSizeInNM = 3.0;
	let v_tempMode = 0;
	let v_contTemp = 300.0;

	let p_kind1, p_kind2, p_nParticles, p_timeStep, p_xMax, p_yMax, p_zMax; // = theModule.getSysParam();
	let sysTime, sysTemp, kineticEnergy, potentialEnergy; // = theModule.getNow();
	let nowData = [];
  let kindList = [];
  let xxList = [];
	let yyList = [];
  let zzList = [];
	let vxList = [];
	let vyList = [];
  let vzList = [];

	let nCalc = 1;
	let dispMode = 0;
	let resetFlag = true;
	let pauseFlag = false;
	let stepFlag = false;

	let breakFlag = false;
	let perticleFlag = true;
	let timeStamp = 0.0;


	function main() {
		resetFlag = true;
		setCanvas( 'canvas_box', xCanvasSize, yCanvasSize );
		initDom();

		animate();

		function setCanvas( canvasID, width, height ) {
			canvas = document.getElementById( canvasID );
			canvas.width  = width;
			canvas.height = height;
			ctx = canvas.getContext('2d');
			ctx.font = "16px 'sans-serif'";
			ctx.textBaseline = "bottom";
			ctx.textAlign = "left";
			ctx.lineWidth = 1;
			g3d.setMouseOnCanvas( canvas ); // 3D graphics
		}
	}


	function animate() {
    if ( breakFlag ) return;

		if ( resetFlag ) {
			resetFlag = false;
			theModule.init( v_theme, v_BoxSizeInNM, v_contTemp ); // ( nn, BoxSizeInNM, contTemp )
			[ p_kind1, p_kind2, p_nParticles, p_timeStep, p_xMax, p_yMax, p_zMax ] = theModule.getSysParam();
			// g3d.init( nParticles, xMax, yMax, zMax, xCanvasSize, yCanvasSize, xBoxSize, yShift )
			g3d.init( p_nParticles, p_xMax, p_yMax, p_zMax, xCanvasSize, yCanvasSize, 300, 20 );
			perticleFlag = true;
		}

		if ( !pauseFlag ) {
			for(let i=0; i<nCalc; i++ ) {
				theModule.evolve( v_tempMode,v_contTemp );
			}
		} else if ( pauseFlag && stepFlag ) {
			stepFlag = false;
			theModule.evolve( v_tempMode, v_contTemp );
		}

		draw( ctx, dispMode );

		if ( perticleFlag ) setParticlesData();

		requestAnimationFrame(animate);
	}

  function setParticlesData() {
		nowData = [ sysTime, sysTemp, kineticEnergy, potentialEnergy ];
		kindList = [];
		xxList = [];
		yyList = [];
		zzList = [];
		vxList = [];
		vyList = [];
		vzList = [];

    for (let i=0; i<p_nParticles; i++) {
			let x, y, z, vx, vy, vz;
      kindList[i] = theModule.getIonData(i)[0];
			[ x, y, z ] = theModule.getPosition(i);
			xxList[i] = x;
			yyList[i] = y;
      zzList[i] = z;
			[ vx, vy, vz ] = theModule.getVelocity(i);
			vxList[i] = vx;
			vyList[i] = vy;
      vzList[i] = vz;
		}
	}


	// --------------------  draw  --------------------

	const ballColorData = [
			/*B */ [ 0xff, 0x40, 0x60 ], /*O */ [ 0x40, 0x00, 0xff ], /*F */ [ 0x00, 0x40, 0xff ],
			/*Na*/ [ 0xff, 0x00, 0x00 ], /*Mg*/ [ 0xff, 0x60, 0x00 ], /*Al*/ [ 0xff, 0x40, 0x80 ],
			/*Si*/ [ 0xff, 0x80, 0x40 ], /*Cl*/ [ 0x00, 0x00, 0xff ], /*K */ [ 0xff, 0x40, 0x00 ],
			/*Ca*/ [ 0xff, 0x80, 0x00 ], /*Ti*/ [ 0xff, 0x80, 0x60 ], /*Sr*/ [ 0xff, 0xaf, 0x00 ],
			/*Ba*/ [ 0xff, 0xbf, 0x00 ], /*He*/ [ 0x00, 0xff, 0x80 ], /*Ne*/ [ 0x00, 0xff, 0x40 ],
			/*Ar*/ [ 0x00, 0xff, 0x00 ], /*Kr*/ [ 0x40, 0xff, 0x00 ], /*Xe*/ [ 0x80, 0xff, 0x00 ] ];

	function draw( ctx, dispMode ) {
		const xBoxSize = 300, xPos=90, yPos = 55, scale = xBoxSize/p_xMax; // g3d param
		const yTextPos= 425;

		ctx.clearRect(0, 0, xCanvasSize, yCanvasSize);

		if ( dispMode==0 ) {
			// draw3D( ctx, nParticles, ballScale, bondFlag )
			draw3D( ctx, p_nParticles, 1.0, false );
		} else if ( dispMode==1 ) {
			draw3D( ctx, p_nParticles, 0.4, false );
		} else if ( dispMode==2 ) {
			draw3D( ctx, p_nParticles, 0.4, true );
		} else if ( dispMode==3 ) {
			draw3D( ctx, p_nParticles, 0.0, true );
		}

		[ sysTime, sysTemp, kineticEnergy, potentialEnergy ] = theModule.getNow();

		ctx.fillStyle = ballColor( p_kind1, 1.0 );
		ctx.fillText( theModule.getKindStr(p_kind1), 20, yTextPos+20);
		ctx.fillStyle = ballColor( p_kind2, 1.0 );
		ctx.fillText(theModule.getKindStr(p_kind2), 20+30, yTextPos+20);

		ctx.fillStyle = "#888888";
		ctx.fillText(`time = ${(sysTime*1.0e12).toFixed(1)} (ps)`, 20, yTextPos);
		ctx.fillText(`Temp = ${sysTemp.toFixed(1)} (K)`, 240, yTextPos);
		ctx.fillText(`, N = ${p_nParticles}`, 80, yTextPos+20);
		ctx.fillText(`control Temp = ${v_contTemp.toFixed(0)} (K)`, 240, yTextPos+20);
		ctx.fillText(`box = ${(p_xMax*1.0e9).toFixed(1)}x${(p_yMax*1.0e9).toFixed(1)}x${(p_zMax*1.0e9).toFixed(1)} (nm)`,
						20, yTextPos+40);
		ctx.fillText(`Energy = ${(kineticEnergy+potentialEnergy).toExponential(4)} (J)`, 240, yTextPos+40);

	}


	function draw3D( ctx, nParticles, ballScale, bondFlag ) {
		let sc, xp, yp; [ sc, xp, yp ] = g3d.scxpypFunc(); // g3d sc, xp, yp
		const nn = nParticles;

		g3d.set3DRotatedObjects(0.0); //(rotateRateOfAyInDegree) eg. 0.0 or 0.5
		g3d.plotFarEdge(ctx, sc,xp,yp,"#444400"); // dark yellow
		for (let i=0; i<nn; i++) {
			const j = g3d.srtzix[i];
			//const zr = (g3d.ppz[j]/g_zMax+1.23)/3.0; // -0.73 < g3d.ppz[j]/g_zMax < 1.73
			const zr = (g3d.ppz[j]/g_zMax+1.0)/2.73; // -0.73 < g3d.ppz[j]/g_zMax < 1.73
			let kind, rCollision, rBond; [ kind, rCollision, rBond ] = theModule.getIonData(j);
			const x = g3d.ppx[j]*sc+xp, y = g3d.ppy[j]*sc+yp, r = rCollision*sc*ballScale;
			if ( r>0.0 ) {
				g3d.drawDisc( ctx, x, y, r, ballColor( kind, zr ) ); // (x, y, r, color)
				g3d.strokeCircle( ctx, x, y, r, ballColor( kind, zr*0.5 ) ); // (x, y, r, color)
			}
			if ( bondFlag ) {
				const reg = theModule.getNearList(j);
				const n = reg[0];
				let x1, y1, z1; [ x1, y1, z1 ] = theModule.getPosition(j);
				for (let k=1; k<n; k++) {
					const jj = reg[k];
					if ( theModule.getChargeProduct(j,jj)>0 ) continue;
					const d0 = theModule.getBondLength(j,jj);
					let x2, y2, z2; [ x2, y2, z2 ] = theModule.getPosition(jj);
					const d = Math.sqrt( (x1-x2)*(x1-x2) + (y1-y2)*(y1-y2) + (z1-z2)*(z1-z2) );
					if ( d<d0*1.2 ) {
						g3d.drawLine(ctx, x, y, g3d.ppx[jj]*sc+xp, g3d.ppy[jj]*sc+yp, "#444444");
					}
				}
			}
		}
		g3d.plotNearEdge( ctx, sc,xp,yp, "#999900" ); // yellow

		ctx.font = "12px 'sans-serif'";
		ctx.fillStyle = "#888888";
		ctx.fillText(`Ax=${(g3d.Ax*180/Math.PI).toFixed(1)}, Ay=${(g3d.Ay*180/Math.PI).toFixed(1)}`, 10, 15);
		ctx.font = "16px 'sans-serif'";
	}

	function ballColor( kind, zr ) {
		const r = Math.floor(ballColorData[kind][0]*zr);
		const g = Math.floor(ballColorData[kind][1]*zr);
		const b = Math.floor(ballColorData[kind][2]*zr);
		return "rgb("+r+","+g+","+b+")";
	}


	// --------------------  3D graphics (particles) module  --------------------
	//
	// ver 0.0.1  2018.12.23  last updated on 2023.04.10
	// ver 0.0.2  2023.05.02  last updated on 2023.06.02

	let g_nParticles, g_xMax, g_yMax, g_zMax, g_xCanvasSize, g_yCanvasSize, g_xBoxSize, g_yShift; // gloval in g3d

	const g3d = {};				// namespace of graphic 3D module
	g3d.mouseDownFlag = 0;		// 1:on mouse down, 0:else
	g3d.x_mouse = 0;			// x-position of mouse
	g3d.y_mouse = 0;			// y-position of mouse
	g3d.x0_mouse = 0;			// drag-started x-position of mouse
	g3d.y0_mouse = 0;			// drag-started y-position of mouse
	g3d.zoom = 1.0;

	g3d.cx0 = 0.0;				// x-rotate center (3D graphics)
	g3d.cy0 = 0.0;				// y-rotate center (3D graphics)
	g3d.cz0 = 0.0;				// z-rotate center (3D graphics)
	g3d.Ax = -Math.PI/12.0;		// x-rotate angle around x-axis
	g3d.Ay = -Math.PI/12.0;		// y-rotate angle around y-axis
	g3d.ddAy = 0.0;				// Ay change rate for auto-rotate: eg. dday=0.5*Math.PI/180
	g3d.cosAx = 0.0;			// cosAx=Math.cos(Ax)
	g3d.sinAx = 0.0;			// sinAx=Math.sin(Ax)
	g3d.cosAy = 0.0;			// cosAy=Math.cos(Ay)
	g3d.sinAy = 0.0;			// sinAy=Math.sin(Ay)

	g3d.ppx = [];				// (m) rotated x-component of i-th particle position
	g3d.ppy = [];				// (m) rotated y-component of i-th particle position
	g3d.ppz = [];				// (m) rotated z-component of i-th particle position
	g3d.srtzix = [];			// z(depth)-sorted index: srtzix[1], srtzix[2],...,srtzix[g_nParticles]

	g3d.xApex = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0];		// x-apex of box
	g3d.yApex = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0];		// y-apex of box
	g3d.zApex = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0];		// z-apex of box
	g3d.pxApex = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0];		// rotated x-apex of box
	g3d.pyApex = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0];		// rotated y-apex of box
	g3d.pzApex = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0];		// rotated z-apex of box
	g3d.boxApex = [[0,0,0], [1,0,0], [0,1,0], [1,1,0], [0,0,1], [1,0,1], [0,1,1], [1,1,1] ];
	g3d.boxEdge = [[0,1,9], [0,2,9], [0,4,9], [1,3,9], [1,5,9], [2,3,9],
				   [2,6,9], [3,7,9], [4,5,9], [4,6,9], [5,7,9], [6,7,9] ];

	//--- set mouse on canvas

	// g3d.setMouseOnCanvas( canvas );
	g3d.setMouseOnCanvas = function( canvas ) {
		canvas.addEventListener('mousemove', g3d.mouse_move);
		canvas.addEventListener('mousedown', g3d.mouse_down);
		canvas.addEventListener('mouseup', g3d.mouse_up);
		//canvas.addEventListener("mousewheel", g3d.mouseWheel);
	};
	g3d.setMouseOnCanvas.defined = false;

	g3d.mouse_move = function(e) {
		const pi = Math.PI;

		if (g3d.mouseDownFlag==1) {
			g3d.x_mouse = e.clientX;
			g3d.y_mouse = e.clientY;
			g3d.Ay = g3d.Ay + 0.5*(g3d.x_mouse-g3d.x0_mouse)*pi/180;
			if (g3d.Ay<-pi) g3d.Ay += 2*pi;
			if (g3d.Ay>pi) g3d.Ay += -2*pi;
			//g3d.Ax = g3d.Ax + 0.5*(g3d.y_mouse-g3d.y0_mouse)*pi/180;
			g3d.Ax = g3d.Ax - 0.5*(g3d.y_mouse-g3d.y0_mouse)*pi/180;
			if (g3d.Ax<-0.5*pi) g3d.Ax = -0.5*pi;
			if (g3d.Ax>0.5*pi) g3d.Ax = 0.5*pi;
			g3d.x0_mouse = g3d.x_mouse;
			g3d.y0_mouse = g3d.y_mouse;
		}
	};

	g3d.mouse_down = function(e) {
		if (g3d.mouseDownFlag==0) {
			g3d.x0_mouse = e.clientX;
			g3d.y0_mouse = e.clientY;
			g3d.x_mouse = g3d.x0_mouse;
			g3d.y_mouse = g3d.y0_mouse;
			g3d.mouseDownFlag = 1;
		}
	};

	g3d.mouse_up = function(e) {
		if (g3d.mouseDownFlag==1) {
			g3d.mouseDownFlag = 0;
		}
	};

	g3d.mouseWheel = function(e) {
		g3d.deltaY = e.deltaY;
		if ( g3d.deltaY > 0 ) g3d.zoom *= 0.95;
		else if ( g3d.deltaY < 0 ) g3d.zoom *= 1.05;
		if ( g3d.zoom<0.5 ) g3d.zoom = 0.5;
		if ( g3d.zoom>2.0 ) g3d.zoom = 2.0;
	};

	//--- 3D graphics aid (particles)

	// g3d.init( nParticles, xMax, yMax, zMax, xCanvasSize, yCanvasSize, xBoxSize, yShift )
	g3d.init = function( nParticles, xMax, yMax, zMax, xCanvasSize, yCanvasSize, xBoxSize, yShift ) {
		g_nParticles = nParticles;
		g_xMax = xMax;
		g_yMax = yMax;
		g_zMax = zMax;
		g_xCanvasSize = xCanvasSize; g_yCanvasSize = yCanvasSize;
		g_xBoxSize = (xBoxSize==undefined) ? 300 : xBoxSize;
		g_yShift = (yShift==undefined) ? 20 : yShift;
		g3d.cx0 = 0.5*g_xMax;		// x-rotate center (3D graphics)
		g3d.cy0 = 0.5*g_yMax;		// y-rotate center (3D graphics)
		g3d.cz0 = 0.5*g_zMax;		// z-rotate center (3D graphics)
	}

	// g3d.setRotateAngle( AxInDegree, AyInDegree );
	g3d.setRotateAngle = function( AxInDegree, AyInDegree ) {
		g3d.Ax = AxInDegree*Math.PI/180.0;
		g3d.Ay = AyInDegree*Math.PI/180.0;
	};

	// g3d.scxpypFunc();
	g3d.scxpypFunc = function() {
		const xBoxSize = g_xBoxSize;
		const xCenter = xCanvasSize/2, yCenter = yCanvasSize/2-g_yShift, yBoxSize = (xBoxSize/g_xMax)*g_yMax;
		const xp = xCenter - (xBoxSize/2)*g3d.zoom, yp = yCenter - (yBoxSize/2)*g3d.zoom; // g3d param
		const sc = (xBoxSize/g_xMax)*g3d.zoom;
		return [ sc, xp, yp ];
	}

	g3d.set3DRotatedObjects = function(rotateRateOfAyInDegree) {
		g3d.ddAy = rotateRateOfAyInDegree*Math.PI/180.0;
		g3d.Ay= g3d.Ay + g3d.ddAy; // auto-rotate : if (ddAy==0.0), stop
		if (g3d.Ay>Math.PI) g3d.Ay = g3d.Ay - 2.0*Math.PI;
		if (g3d.Ay<-Math.PI) g3d.Ay = g3d.Ay + 2.0*Math.PI;
		g3d.setBox();           // set box apex
		g3d.setRotateXY(g3d.Ax,g3d.Ay); // set rotate angle and rotate center(=box center)
		g3d.rotateXY();         // g_xx(i),g_yy(i),g_zz(i) rotate--> ppx(i),ppy(i),ppz(i)
		g3d.sortz();            // sort ppz(i) : ppz(srtzix(1)),ppz(srtzix(2)),...,ppz(srtzix(g_nParticles))
		g3d.markFarEdge();      // boxEdge[iEdge][2]=1:far side edge or 0:near side edge
	};

	g3d.setBox = function() {
		for (let i=0; i<8; i++) {
			g3d.xApex[i] = g3d.boxApex[i][0]*g_xMax;
			g3d.yApex[i] = g3d.boxApex[i][1]*g_yMax;
			g3d.zApex[i] = g3d.boxApex[i][2]*g_zMax;
		}
	};

	g3d.setRotateXY = function(angleX,angleY) {
		g3d.cosAx = Math.cos(angleX);
		g3d.sinAx = Math.sin(angleX);
		g3d.cosAy = Math.cos(angleY);
		g3d.sinAy = Math.sin(angleY);
		g3d.cx0 = 0.5*g_xMax;
		g3d.cy0 = 0.5*g_yMax;
		g3d.cz0 = 0.5*g_zMax;
	};

	g3d.rotateXY = function() { //particles and box apex
		const nn = g_nParticles;
		const cosAx=g3d.cosAx,sinAx=g3d.sinAx,cosAy=g3d.cosAy,sinAy=g3d.sinAy,cx0=g3d.cx0,cy0=g3d.cy0,cz0=g3d.cz0;

		for (let i=0; i<nn; i++) {
			let xi, yi, zi;
			[ xi, yi, zi ] = theModule.getPosition(i);
			const y = g_yMax-yi-cy0;
			g3d.ppx[i] = cosAy*(xi-cx0)+sinAy*(sinAx*y+cosAx*(zi-cz0)) + cx0;
			g3d.ppy[i] = cosAx*y-sinAx*(zi-cz0) + cy0;
			g3d.ppz[i] =-sinAy*(xi-cx0)+cosAy*(sinAx*y+cosAx*(zi-cz0)) + cz0;
		}
		for (let i=0; i<8; i++) {
			g3d.pxApex[i] = cosAy*(g3d.xApex[i]-cx0)+sinAy*(sinAx*(g3d.yApex[i]-cy0)+cosAx*(g3d.zApex[i]-cz0))+cx0;
			g3d.pyApex[i] = cosAx*(g3d.yApex[i]-cy0)-sinAx*(g3d.zApex[i]-cz0) + cy0;
			g3d.pzApex[i] =-sinAy*(g3d.xApex[i]-cx0)+cosAy*(sinAx*(g3d.yApex[i]-cy0)+cosAx*(g3d.zApex[i]-cz0))+cz0;
		}
	};

	g3d.rotateXYPos = function(xi,yi,zi) {
		const cosAx=g3d.cosAx,sinAx=g3d.sinAx,cosAy=g3d.cosAy,sinAy=g3d.sinAy,cx0=g3d.cx0,cy0=g3d.cy0,cz0=g3d.cz0;
		const y = g_yMax-yi-cy0;
		const rotx = cosAy*(xi-cx0)+sinAy*(sinAx*y+cosAx*(zi-cz0)) + cx0;
		const roty = cosAx*y-sinAx*(zi-cz0) + cy0;
		//const rotz =-sinAy*(xi-cx0)+cosAy*(sinAx*y+cosAx*(zi-cz0)) + cz0;
		return [ rotx, roty ];
	};

	g3d.sortz = function() {
		const nn=g_nParticles;
		for (let i=0; i<nn; i++) {
			g3d.srtzix[i] = i;
		}
		g3d.qSort(0,nn-1);
	};

	g3d.qSort = function(le,ri) {
		let i,j, pv,w;
		if (ri>le) {
			i = le-1;
			j = ri;
			pv = g3d.ppz[g3d.srtzix[ri]];
			while (1) {
				do {
					i=i+1;
				} while (pv>g3d.ppz[g3d.srtzix[i]]);
				do {
					j=j-1;
				} while (j>i && g3d.ppz[g3d.srtzix[j]]>pv);
				if (j<=i) break;
				w=g3d.srtzix[i]; g3d.srtzix[i]=g3d.srtzix[j]; g3d.srtzix[j]=w;
			}
			w=g3d.srtzix[i]; g3d.srtzix[i]=g3d.srtzix[ri]; g3d.srtzix[ri]=w;
			g3d.qSort(le,i-1);
			g3d.qSort(i+1,ri);
		}
	};

	g3d.markFarEdge = function() {
		//seek far apex --> iMin
		let zMin = g3d.pzApex[0];
		let iMin = 0;
		for (let i=1; i<8; i++) {
			if (zMin>g3d.pzApex[i]) {
				zMin = g3d.pzApex[i];
				iMin = i;
			}
		}
		//mark far edge
		for (let iEdge=0; iEdge<12; iEdge++) {
			g3d.boxEdge[iEdge][2] = 0;
			if (g3d.boxEdge[iEdge][0]==iMin || g3d.boxEdge[iEdge][1]==iMin) g3d.boxEdge[iEdge][2] = 1;
		}
	};

	g3d.plotNearEdge = function(ctx, sc,xp,yp,color) {
		for (let iEdge=0; iEdge<12; iEdge++) {
			if (g3d.boxEdge[iEdge][2]==0) { //far edge mark == 1
				g3d.plotEdge(ctx, iEdge,sc,xp,yp,color);
			}
		}
	};

	g3d.plotFarEdge = function(ctx, sc,xp,yp,color) {
		for (let iEdge=0; iEdge<12; iEdge++) {
			if (g3d.boxEdge[iEdge][2]==1) { //far edge mark == 1
				g3d.plotEdge(ctx, iEdge,sc,xp,yp,color);
			}
		}
	};

	g3d.plotEdge = function(ctx, iEdge,sc,xp,yp,color) {
		let iApex = g3d.boxEdge[iEdge][0];
		const x1 = g3d.pxApex[iApex]*sc+xp;
		const y1 = g3d.pyApex[iApex]*sc+yp;
		iApex = g3d.boxEdge[iEdge][1];
		const x2 = g3d.pxApex[iApex]*sc+xp;
		const y2 = g3d.pyApex[iApex]*sc+yp;
		g3d.drawLine(ctx, x1, y1, x2, y2, color);
	};

	g3d.drawLine = function(ctx, x1, y1, x2, y2, color) {
		ctx.strokeStyle = color;
		ctx.beginPath();
		ctx.moveTo(x1, y1);
		ctx.lineTo(x2, y2);
		ctx.stroke();
	};

	g3d.drawDisc = function( ctx, x, y, r, color ) {
		ctx.fillStyle = color;
		ctx.beginPath();
		ctx.arc(x, y, r, 0, 2*Math.PI, false);
		ctx.fill();
	};

	g3d.strokeCircle = function( ctx, x, y, r, color ) {
		ctx.strokeStyle = color;
		ctx.beginPath();
		ctx.arc(x, y, r, 0, 2*Math.PI, false);
		ctx.stroke();
	}

	// --------------------  3D graphics (particles) module end  --------------------


	// --------------------  dom control

	function initDom() {
		document.getElementById("step_button").style.visibility = "hidden";
	}

	function reset() { resetFlag = true; }

	function pause() {
		let btn = document.getElementById("pause_button");

		pauseFlag = ( pauseFlag==false );
		if ( pauseFlag==false ) btn.innerHTML = "pause"; else btn.innerHTML = "go";

		if ( pauseFlag==true ) {
			document.getElementById("step_button").style.visibility = "visible";
		} else {
			document.getElementById("step_button").style.visibility = "hidden";
		}
	}

	function step() { stepFlag = true; }

	function setTheme() {
		v_theme = 0 + document.getElementById("slct_theme").selectedIndex;
		resetFlag = true;
	}

	function setnParticles() {
		const n = 1 + document.getElementById("slct_nn").selectedIndex;
		v_nn = n*100;
		resetFlag = true;
	}

	function setTempMode() {
		v_tempMode = 0 + document.getElementById("slct_tempMode").selectedIndex;
	}

	function setContTemp() {  // range mixing
		v_contTemp = Number(document.getElementById("range_temp").value);
		document.getElementById("text_temp").innerHTML = " " + v_contTemp.toFixed(0);
	}

	function setDispMode() {
		dispMode = 0 + document.getElementById("slct_dispMode").selectedIndex;
	}

	function setNcalc() {
		nCalc = 1 + document.getElementById("slct_nCalc").selectedIndex;
	}

	function viewHome() {
		g3d.setRotateAngle(-15,-15);
		g3d.zoom = 1.0;
	}

  // function controlled by python

  function breakLoop() {
    breakFlag = true;
  }

  function pysetTheme( theme ) {
    v_theme = theme
    document.getElementById("slct_theme").selectedIndex = theme;
    resetFlag = true;
  }

  function pysetTempMode( mode ) {
    v_tempMode = mode;
    document.getElementById("slct_tempMode").selectedIndex = mode;
  }

  function pysetTemperature(temp) {
    theModule.setTemp(temp);
  }

  function pysetDispMode( mode ) {
    dispMode = mode;
    document.getElementById("slct_dispMode").selectedIndex = mode;
  }

  function pygetData( pyMsg ) {
    document.getElementById("text_from_python").innerHTML = pyMsg;
    return [ sysTime, sysTemp, kineticEnergy, potentialEnergy ];
  }

  function pysetDispState( ist ) {
    dispState = ist;
    document.getElementById("slct_dispState").selectedIndex = ist;
  }

  function pygetParticlesList() {
    perticleFlag = false;
    return [ nowData, kindList, xxList, yyList, zzList, vxList, vyList, vzList ];
  }


	// --------------------  public  --------------------

	return {
		main:			main,			// main()

		// dom control
		reset:			reset,			// reset()
		pause:			pause,			// pause()
		step:			step,			// step()

		setTheme:		setTheme,		// setTheme()
		setnParticles:	setnParticles,	// setnParticles()
		setTempMode:	setTempMode,	// setTempMode()
		setContTemp:	setContTemp,	// setContTemp()
		setDispMode:	setDispMode,	// setDispMode()
		setNcalc:		setNcalc,		// setNcalc()
		viewHome:		viewHome,		// viewHome()

    breakLoop: breakLoop, // breakLoop();
    pysetTheme: pysetTheme, // pysetTheme( theme )
    pysetTempMode: pysetTempMode, // pysetTempMode( mode )
    pysetTemperature: pysetTemperature, // pysetTemperature( temp )
    pysetDispMode: pysetDispMode, // pysetDispMode( mode )
    pygetData: pygetData, // pygetData( pyMsg ) : return [ sysTime, sysTemp, kineticEnergy, potentialEnergy ]
		pygetParticlesList: pygetParticlesList, //() :return [ nowData, kindList, xxList, yyList, zzList, vxList, vyList, vzList ]
	};


})(); // ====================  js027 module end  ====================


const js = js027;
//window.addEventListener('load', js.main );
js.main();


// %%%%%%%%%%%%%%%%%%%%  end of javaScript  %%%%%%%%%%%%%%%%%%%%

</script>
<style type="text/css">
body { text-align:left; color:#000000; background-color:#fff8dd; }
</style>
</head>

<body>
<p>[js027] ionic crystal - ion MD3D</p>
<canvas ID="canvas_box" style="background-color: #000000;" WIDTH="480" HEIGHT="480"></canvas><br>

<label>material:</label>
<select id="slct_theme" onChange="js.setTheme()">
<option>NaCl</option><option>MgO</option><option>CaO</option><option>BaO</option>
<option>NaF</option><option>KF</option><option>KCl</option>
</select>
    <span style="margin-right: 180px;"></span>
<button onClick="js.reset()">reset</button>
    <span style="margin-right: 20px;"></span>
<button id="pause_button" onClick="js.pause()">pause</button>
    <span style="margin-right: 10px;"></span>
<button id="step_button" onClick="js.step()">step</button>
<br>

<label>temp mode:</label>
<select id="slct_tempMode" onChange="js.setTempMode()">
<option selected>adiabatic</option><option>Temp.control</option>
</select>
<br>

<label>cont. Temp</label><label id="text_temp"> 300</label>
<input type="range" id="range_temp" min="10" max="2500" value="300" step="2"
style="width:360px" oninput="js.setContTemp()">
<br>

<label> disp. mode:</label>
<select id="slct_dispMode" onChange="js.setDispMode()">
<option selected>ball</option><option>small ball</option>
<option>small ball + bond</option><option>bond only</option>
</select>
    <span style="margin-right: 100px;"></span>
<button onClick="js.viewHome()">return to initial view</button>
<br>

<p id="text_caption" ></p>
<hr width="400" align="left" color="#a0a0a0">
<button onClick="js.breakLoop()">animation break to END</button>
    <span style="margin-right: 50px;"></span> python msg:
<span id="text_from_python" ></span>
<br>

</body>
</html>


    ''')
  display(htm)
# end def


In [ ]:
# exec html-js code
exec_html_js()
print("--- push [animation break to END] button to end ---")

In [ ]:
# get data and print

import time

# simulator run
exec_html_js()
print("-- start --")

# get data and print
for i in range(10):
  # get data and display
  [ sysTime, sysTemp, ke, pe ] = eval_js( 'js.pygetData({})'.format(i) )
  energy = ke + pe
  print(f'i = {i:>2d},  time = {sysTime*1e12:>7.2f} (ps),  temp = {sysTemp:>6.1f} (K),  K = {ke:9.6e},  U = {pe:9.6e},  energy = {energy:9.6e} (J)')
  time.sleep(1)

# simulator stop
eval_js( 'js.breakLoop()' )
print("-- stop --")


In [ ]:
# change theme, dispMode

import time

themeList = [ '0:NaCl', '1:MgO', '2:CaO','3:BaO', '4:NaF', '5:KF', '6:KCl' ]
dispModeList = [ '0: ball', '1: small ball', '2: small ball + bond', '3: bond only' ]

# simulator run
exec_html_js()
print("-- start --")

# change theme and dispMode
for theme in [ 0, 4, 5, 6 ]:
  eval_js( 'js.pysetTheme({})'.format(theme) )
  print( "-- theme:", themeList[theme], "--" )
  for dispMode in [ 0, 1, 2, 3 ]:
    eval_js( 'js.pysetDispMode({})'.format(dispMode) )
    print( "   -- dispMode:", dispModeList[dispMode], "--" )
    # get data and display
    [ sysTime, sysTemp, ke, pe ] = eval_js( 'js.pygetData(0)' )
    energy = ke + pe
    print(f'\t time = {sysTime*1e12:>7.2f} (ps),  temp = {sysTemp:>6.1f} (K),  K = {ke:9.6e},  U = {pe:9.6e},  energy = {energy:9.6e} (J)')
    time.sleep(3)

# simulator stop
eval_js( 'js.breakLoop()' )
print("-- stop --")

In [ ]:
# system heat up and get data (timeList, tempList, energyList) and plot

import time
import matplotlib.pyplot as plt

# simulator run
exec_html_js()
print("-- start --")

# change dispMode
eval_js( 'js.pysetDispMode(2)' ) # 2:'small ball + bond'
print("-- change dispMode=2:'small ball + bond' --")

timeList = []
tempList = []
energyList = []

for i in range(20):
  # set temperature
  temp = 300 + 100*i
  eval_js( 'js.pysetTemperature({})'.format(temp) )
  time.sleep(3)

  # get data and display
  [ sysTime, sysTemp, ke, pe ] = eval_js( 'js.pygetData({})'.format(i) )
  energy = ke + pe

  timeList.append(sysTime*1e12)
  tempList.append(sysTemp)
  energyList.append(energy*1e20)

  if i%2==0:
    print( f'\t time = {sysTime*1e12:>7.2f} (ps),  temp = {sysTemp:>6.1f} (K),  energy = {energy:9.6e} (J)' )
  time.sleep(3)

# simulator stop
eval_js( 'js.breakLoop()' )
print("-- stop --")

# plot tempList
plt.plot( timeList, tempList, 'og-' )
plt.xlabel('time (ps)')
plt.ylabel('temp (K)')
plt.show()

# plot energyList
plt.plot( timeList, energyList, 'xr-' )
plt.xlabel('time (ps)')
plt.ylabel('temp (K)')
plt.show()

In [ ]:
# colab AI wrote:
# prompt: plot tempList, energyList in one figure with dual y-axis

import matplotlib.pyplot as plt

# plot tempList and energyList in one figure with dual y-axis
fig, ax1 = plt.subplots()

ax1.plot(timeList, tempList, 'og-', label='Temperature')
ax1.set_ylabel('Temperature (K)', color='g')
ax1.tick_params(axis='y', labelcolor='g')

ax2 = ax1.twinx()
ax2.plot(timeList, energyList, 'xr-', label='Energy')
ax2.set_ylabel('Energy (J)', color='r')
ax2.tick_params(axis='y', labelcolor='r')

plt.xlabel('Time (ps)')
plt.title('Temperature and Energy vs. Time')
plt.legend()
plt.show()


In [ ]:
# get particles data and save

import time
import numpy as np

themeList = [ '0:NaCl', '1:MgO', '2:CaO','3:BaO', '4:NaF', '5:KF', '6:KCl' ]

# simulator run
exec_html_js()
print("-- start --")

# set theme
theme = 0 # '0:NaCl'
eval_js( 'js.pysetTheme({})'.format(theme) )
print( "-- theme:", themeList[theme], "--" )

# wait 5 sec.
time.sleep(5)

# get data and print
[sysTime, sysTemp, ke, pe ] = eval_js('js.pygetData({})'.format(0))
energy = ke + pe
print( f'  time = {sysTime*1e12:>7.2f} (ps),  temp = {sysTemp:>6.1f} (K),  energy = {energy:9.6e} (J)' )

# get particle data and plot
print("-- get particles data --")
[ nowData, kindList, xxList, yyList, zzList, vxList, vyList, vzList ] = eval_js('js.pygetParticlesList()')
[sysTime, sysTemp, ke, pe ] = nowData
energy = ke + pe
print(f'got time = {sysTime*1e12:>7.2f} (ps),  temp = {sysTemp:>6.1f} (K),  energy = {energy:9.6e} (J)')
print("-- get kindList, xxList, yyList, zzList, vxList, vyList, vzList,  length of list = ", len(vxList))

# simulator stop
eval_js( 'js.breakLoop()' )
print("-- simulator stop --")

# convert [ kindList, xxList, yyList, zzList, vxList, vyList, vzList ] to np_data
np_data = np.array([ kindList, xxList, yyList, zzList, vxList, vyList, vzList ])
print("shape of np_data :", np_data.shape )

# save np_data
print("-- save as 'js027_data.npy'" )
np.save( 'js027_data.npy', np_data )

# save np_data.T as csv file
#np.savetxt('js027_data.csv', np_data.T, delimiter=',')

In [ ]:
# load particles data

import numpy as np

ionList =     [ "B",   "O",  "F",  "Na", "Mg", "Al", "Si", "Cl", "K", "Ca", "Ti", "Sr", "Ba" ]
SizeOf = np.array([ 0.23, 1.40, 1.33, 1.02, 0.72, 0.53, 0.40, 1.81, 1.38, 1.00, 0.61, 1.16, 1.36 ]) # bond radius (x 1e-10 m)
ColorOf = np.array([
    [0xff, 0x40, 0x60], [0x40, 0x00, 0xff], [0x00, 0x40, 0xff], [0xff, 0x00, 0x00],  # B, O, F, Na
    [0xff, 0x60, 0x00], [0xff, 0x40, 0x80], [0xff, 0x80, 0x40], [0x00, 0x00, 0xff],  # Mg, Al, Si, Cl
    [0xff, 0x40, 0x00], [0xff, 0x80, 0x00], [0xff, 0x80, 0x60], [0xff, 0xaf, 0x00], [0xff, 0xbf, 0x00]  # K, Ca, Ti, Sr, Ba
]) / 255

x_box_size, y_box_size, z_box_size = 3.0, 3.0, 3.0

# load particles data
loaded_data = np.load('js027_data.npy')
print(f"-- load particles data  shape:{loaded_data.shape} ")

# set numpy array
Kind = loaded_data[0].astype(int)  # kind of every atom
X = loaded_data[1] * 1e9           # (nm) x-position of every atom
Y = loaded_data[2] * 1e9           # (nm) y-position of every atom
Z = loaded_data[3] * 1e9           # (nm) z-position of every atom
U = loaded_data[4]                 # (m/s) x-velocity of every atom
V = loaded_data[5]                 # (m/s) y-velocity of every atom
W = loaded_data[6]                 # (m/s) y-velocity of every atom
print(f'shape of Kind={Kind.shape}, X={X.shape}, Y={Y.shape}, Z={Z.shape}, U={U.shape}, V={V.shape}, W={W.shape}')
Vabs = np.sqrt(U**2 + V**2 + W**2) # (m/s) speed of every atom
C = ColorOf[Kind]                  # color of every atom
S = SizeOf[Kind] * 2.0             # size of every atom ( diameter in 1e-10 m = 0.1 nm )

In [ ]:
# Na Cl in the box

import numpy as np
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter3d(
    x=X, y=Y, z=Z,
    mode='markers',
    marker=dict(
        color=C,
        size=S*5,
        opacity=0.2,
    )
))

fig.update_layout(
    title='NaCl velocity space ( red: Na, blue: Cl )',
    width=800, height=800,
    scene=dict(
        xaxis_title='x', yaxis_title='y', zaxis_title='z',
        xaxis=dict(range=(0, 3)),
        yaxis=dict(range=(0, 3)),
        zaxis=dict(range=(0, 3)),
    )
)

fig.show()

# save this figure as html file
#fig.write_html('NaCl_in_the_box.html')

In [ ]:
# velocity space

import numpy as np
import plotly.graph_objects as go

vmax = 800 #(m/s)

fig = go.Figure()
fig.add_trace(go.Scatter3d(
    x=U, y=V, z=W,
    mode='markers',
    marker=dict(
        color=C,
        size=S*2,
        opacity=0.3,
    )
))

fig.update_layout(
    title='NaCl velocity space ( red: Na, blue: Cl )',
    width=800, height=800,
    scene=dict(
        xaxis_title='vx', yaxis_title='vy', zaxis_title='vz',
        xaxis=dict(range=(-vmax, vmax)),
        yaxis=dict(range=(-vmax, vmax)),
        zaxis=dict(range=(-vmax, vmax)),
        camera=dict(
            eye=dict(x=1.0, y=-2.0, z=1.0),  # position of camera
            center=dict(x=0, y=0, z=0),  # camera look at
            up=dict(x=0, y=0, z=1)  # upside vector
        ),
    )
)

fig.show()

# save this figure as html file
#fig.write_html('velocity_space_plot.html')

In [ ]:
# hist Vabs

import numpy as np
import matplotlib.pyplot as plt

# get Na(kind==3), Cl(kind==7) data
Na_Vabs = Vabs[Kind==3]
Cl_Vabs = Vabs[Kind==7]

# plot
fig = plt.figure(figsize=(6, 6))
plt.hist(Na_Vabs, bins=30, color='red', alpha=0.5, label='Na')
plt.hist(Cl_Vabs, bins=30, color='blue', alpha=0.5, label='Cl')
plt.xlabel('speed (m/s)')
plt.ylabel('counts')
plt.legend()
plt.show()

In [ ]:
# bond (Na-Cl) length distribution

import numpy as np
import matplotlib.pyplot as plt

# normal bond length of Na - Cl
normal_bond_length = (SizeOf[3] + SizeOf[7]) * 0.1

# bond length distribution
bond_length_list = []
for i in range(len(X)):
  for j in range(i+1, len(X)):
    dx = X[i] - X[j]
    dy = Y[i] - Y[j]
    dz = Z[i] - Z[j]
    dr = np.sqrt(dx**2 + dy**2 + dz**2)
    if (dr < normal_bond_length * 1.2) and (Kind[i] != Kind[j]):
      bond_length_list.append(dr)

# plot bond length distribution
plt.hist(bond_length_list, bins=50)
plt.title('Na-Cl bond length distribution')
plt.xlabel('bond length (nm)')
plt.ylabel('frequency')
plt.show()

In [ ]:
# prompt: plot bonds as scatter points (xj-xi, yj-yi, zj-zi)

import numpy as np
import plotly.graph_objects as go

bond_dx_list = []
bond_dy_list = []
bond_dz_list = []

for i in range(len(X)):
  for j in range(i+1, len(X)):
    dx = X[i] - X[j]
    dy = Y[i] - Y[j]
    dz = Z[i] - Z[j]
    dr = np.sqrt(dx**2 + dy**2 + dz**2)
    # Check if the distance is within 1.2 times the normal bond length and the particles are different kinds (Na and Cl)
    if (dr < normal_bond_length * 1.2) and (Kind[i] != Kind[j]):
        bond_dx_list.append(dx)
        bond_dy_list.append(dy)
        bond_dz_list.append(dz)

fig = go.Figure()
fig.add_trace(go.Scatter3d(
    x=bond_dx_list, y=bond_dy_list, z=bond_dz_list,
    mode='markers',
    marker=dict(
        size=5,
        opacity=0.5,
    )
))

fig.update_layout(
    title='Na-Cl bond displacement vector distribution',
    width=800, height=800,
    scene=dict(
        xaxis_title='dx (nm)', yaxis_title='dy (nm)', zaxis_title='dz (nm)',
        xaxis=dict(range=(-normal_bond_length*1.2, normal_bond_length*1.2)),
        yaxis=dict(range=(-normal_bond_length*1.2, normal_bond_length*1.2)),
        zaxis=dict(range=(-normal_bond_length*1.2, normal_bond_length*1.2)),
        camera=dict(
            eye=dict(x=1.0, y=-2.0, z=1.0),  # position of camera
            center=dict(x=0, y=0, z=0),  # camera look at
            up=dict(x=0, y=0, z=1)  # upside vector
        ),
    )
)

fig.show()


In [ ]:
# plot bond line direction and length
# prompt: from bond_dx_list, bond_dy_list, bond_dz_list above,  plot lines (0,0,0)-(dx, dy, dz)

import numpy as np
import plotly.graph_objects as go

fig = go.Figure()

# Add lines from (0,0,0) to (dx, dy, dz) for each bond
for dx, dy, dz in zip(bond_dx_list, bond_dy_list, bond_dz_list):
    fig.add_trace(go.Scatter3d(
        x=[0, dx],
        y=[0, dy],
        z=[0, dz],
        mode='lines',
        line=dict(
            color='blue',
            width=2
        )
    ))

fig.update_layout(
    title='Na-Cl bond displacement vectors',
    width=800, height=800,
    scene=dict(
        xaxis_title='dx (nm)', yaxis_title='dy (nm)', zaxis_title='dz (nm)',
        xaxis=dict(range=(-normal_bond_length * 1.2, normal_bond_length * 1.2)),
        yaxis=dict(range=(-normal_bond_length * 1.2, normal_bond_length * 1.2)),
        zaxis=dict(range=(-normal_bond_length * 1.2, normal_bond_length * 1.2)),
        camera=dict(
            eye=dict(x=1.0, y=-2.0, z=1.0),  # position of camera
            center=dict(x=0, y=0, z=0),  # camera look at
            up=dict(x=0, y=0, z=1)  # upside vector
        ),
    ),
    showlegend=False # Hide the legend as each line is a separate trace
)

fig.show()


In [ ]:
# data analysis with pandas

import numpy as np
import pandas as pd

# calculate correlation matrix
corr = np.corrcoef(np.array([X, Y, Z, U, V, W]))
print(corr)

# print 1/sqrt(N)
print("1/sqrt(N) = ",1 / np.sqrt(len(X)))

# create a scatter matrix
pd.plotting.scatter_matrix(pd.DataFrame({'X': X, 'Y': Y, 'Z': Z, 'U': U, 'V': V, 'W': W}))

In [ ]:
# plot ions with bond lines
# colab AI wrote:
# prompt: plot Na-Cl bond in the figure of 'NaCl in the box'

fig = go.Figure()

# add Na atoms
fig.add_trace(go.Scatter3d(
    x=X[Kind==3], y=Y[Kind==3], z=Z[Kind==3],
    mode='markers',
    marker=dict(
        color=C[Kind==3],
        size=S[Kind==3]*5,
        opacity=0.2,
    )
))

# add Cl atoms
fig.add_trace(go.Scatter3d(
    x=X[Kind==7], y=Y[Kind==7], z=Z[Kind==7],
    mode='markers',
    marker=dict(
        color=C[Kind==7],
        size=S[Kind==7]*5,
        opacity=0.2,
    )
))

# add Na-Cl bonds
for i in range(len(X)):
    if Kind[i] == 3:  # Na atom
        for j in range(len(X)):
            if Kind[j] == 7:  # Cl atom
                dx = X[i] - X[j]
                dy = Y[i] - Y[j]
                dz = Z[i] - Z[j]
                dr = np.sqrt(dx**2 + dy**2 + dz**2)
                if dr < normal_bond_length * 1.2:
                    fig.add_trace(go.Scatter3d(
                        x=[X[i], X[j]],
                        y=[Y[i], Y[j]],
                        z=[Z[i], Z[j]],
                        mode='lines',
                        line=dict(
                            color='black',
                            width=2,
                        )
                    ))

fig.update_layout(
    title='NaCl in the box with Na-Cl bonds',
    width=800, height=800,
    scene=dict(
        xaxis_title='x', yaxis_title='y', zaxis_title='z',
        xaxis=dict(range=(0, 3)),
        yaxis=dict(range=(0, 3)),
        zaxis=dict(range=(0, 3)),
    )
)

fig.show()
